In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487-final-project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

['checkpoint_7.pth', 'checkpoint_no_na.pth', 'nb.ipynb', '__pycache__', 'data', 'requirements.txt', 'preprocess.py', 'test.py', 'eval.py', 'README.md', 'naivebayes.py', 'scrape.py', 'checkpoint.pth', 'model.py', 'train_model.ipynb']


In [3]:
!pip install transformers
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from google.colab import files

     |████████████████████████████████| 3.8 MB 8.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 54.1 MB/s 
     |████████████████████████████████| 895 kB 54.2 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

In [4]:
from model import train
from model import BertClassifier

# Dataset

In [5]:
np.random.seed(112)

df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'data/MBIC/labeled_dataset.csv'))

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

# Hyperparameter Tuning

In [19]:
# Hyperparameter Tuning Helper Function
from model import Dataset
def hyperparameter_tuning_helper(model_val, batch_size):
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")

  # model_val = BertClassifier()
  # check what checkpoint you need to load from
  # state_dict = torch.load(os.path.join(GOOGLE_DRIVE_PATH, checkpoint_path))
  # model_val.load_state_dict(state_dict)
  # model_val.to(device)

  validation_data = Dataset(df_val)
  val_dataloader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=True)

  total_acc_val = 0
  with torch.no_grad():
    for val_input, val_label in tqdm(val_dataloader):

        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model_val(input_id, mask)
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
        
  print()
  print("Validation Accuracy of", total_acc_val/len(df_val))
  return total_acc_val

Hyperparameter Tuning on Validation Set

In [20]:
from transformers.utils.dummy_pt_objects import BeitForMaskedImageModeling
# epoch_list = [3, 5, 10, 15, 20]
learning_rate_list = [1e-6, 1e-5, 1e-4]
hidden_sizes = [128, 256, 512]
batch_sizes = [16, 8, 4, 2]

model_iteration = 0
best_validation_accuracy = 0
best_hyperparameters = {
                            "learning_rate": None,
                            "hidden_size": None,
                            "batch_size": None
                            }

epoch_value = 5
# for epoch_value in epoch_list:
for hidden_size in hidden_sizes:
  for learning_rate in learning_rate_list:
    for batch_size in batch_sizes:
      print(hidden_size, learning_rate, batch_size)
      bert_clf = BertClassifier()
      # checkpoint_string = "checkpoint_hyperparameter_tuning_" + str(model_iteration) + ".pth"
      train(bert_clf, df_train, df_val, learning_rate, epoch_value)
      # torch.save(bert_clf.state_dict(), os.path.join(GOOGLE_DRIVE_PATH, checkpoint_string))
      # files.download(os.path.join(GOOGLE_DRIVE_PATH, checkpoint_string))
      # test_model_helper(checkpoint_string)
      # state_dict = bert_clf.state_dict()

      current_val_accuracy = hyperparameter_tuning_helper(bert_clf, batch_size)

      if current_val_accuracy > best_validation_accuracy:
          best_validation_accuracy = current_val_accuracy
          best_hyperparameters["hidden_size"] = hidden_size
          best_hyperparameters["learning_rate"] = learning_rate
          best_hyperparameters["batch_size"] = batch_size

      model_iteration += 1

print("The highest accuracy achieved by the model was:", best_validation_accuracy)
print("Best Hyperparameters:", best_hyperparameters)

128 1e-06 16


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using Cuda: True


100%|██████████| 680/680 [01:32<00:00,  7.34it/s]


Epochs: 1 | Train Loss:  0.540             | Train Accuracy:  0.433             | Val Loss:  0.533             | Val Accuracy:  0.476


...

Epochs: 4 | Train Loss:  0.549             | Train Accuracy:  0.404             | Val Loss:  0.549             | Val Accuracy:  0.388


100%|██████████| 680/680 [01:32<00:00,  7.34it/s]


Epochs: 5 | Train Loss:  0.549             | Train Accuracy:  0.404             | Val Loss:  0.549             | Val Accuracy:  0.388


100%|██████████| 85/85 [00:03<00:00, 23.89it/s]


Validation Accuracy of 0.38823529411764707
The highest accuracy achieved by the model was: 160
Best Hyperparameters: {'learning_rate': 1e-05, 'hidden_size': 128, 'batch_size': 8}


In [ ]:
# best_validation_accuracy = 0.9411764705882353

# 128 1e-05 4
# Epoch 3
# 0.953
# {'Epochs': 3, 'hidden_layer_size': 128, 'learning_rate': 1e-05, 'batch_size': 4}



# Train Model

In [8]:
num_epochs = 3
learning_rate = 1e-5
hidden_size = 128
batch_size = 4
clf = BertClassifier(hidden_size=hidden_size)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
train(clf, df_train, df_val, learning_rate, num_epochs, batch_size=batch_size)

torch.save(clf.state_dict(), os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_tuned.pth'))

files.download(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_tuned.pth'))

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Using Cuda: True



100%|██████████| 340/340 [02:06<00:00,  2.68it/s]


Epochs: 1 | Train Loss:  0.229             | Train Accuracy:  0.560             | Val Loss:  0.186             | Val Accuracy:  0.688


...

Epochs: 2 | Train Loss:  0.127             | Train Accuracy:  0.823             | Val Loss:  0.098             | Val Accuracy:  0.894



100%|██████████| 340/340 [02:14<00:00,  2.53it/s]


Epochs: 3 | Train Loss:  0.058             | Train Accuracy:  0.935             | Val Loss:  0.083             | Val Accuracy:  0.912


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test Model

In [19]:
from model import Dataset
from eval import Evaluation

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model_test = BertClassifier(hidden_size=hidden_size)
state_dict = torch.load(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_tuned.pth'))
model_test.load_state_dict(state_dict)
model_test.to(device)

test_data = Dataset(df_test)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

total_acc_test = 0
with torch.no_grad():
    preds = []
    labels = []
    for test_input, test_label in tqdm(test_dataloader):
        test_label = test_label.to(device)
        mask = test_input['attention_mask'].to(device)
        input_id = test_input['input_ids'].squeeze(1).to(device)

        output = model_test(input_id, mask)
        acc = (output.argmax(dim=1) == test_label).sum().item()
        preds.extend(output.argmax(dim=1).tolist())
        labels.extend(test_label.tolist())
        total_acc_test += acc

eval = Evaluation(preds, labels)
eval.all_metrics()


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 43/43 [00:05<00:00,  8.02it/s]


Accuracy: 0.9058823529411765
-----------------------
Macro
F1 score macro: 0.91614417989418
Precision macro: 0.9156813063063063
Recall macro: 0.9168612054329373
-----------------------
Micro
F1 score micro: 0.9058823529411765
Precision micro: 0.9058823529411765
Recall micro: 0.9058823529411765
